In [2]:
import re

import pandas as pd
import os

from sqlalchemy import create_engine, Column, Integer, Float, String, ForeignKey, UniqueConstraint, func
from sqlalchemy.orm import DeclarativeBase, Session

from dotenv import dotenv_values

from municipio_id import municipio_map

## DATA FILTER CLASS

- transformar tudo para o tipo de dado do banco de dados, se possivel com SQLALCHEMY

In [3]:
class InepFilters():
    # csv_path is path source folder + file name
    def __init__(self, csv_path) -> None:
        self.__file_path = csv_path
        source_path, self.__file_name = os.path.split(csv_path)
        if '__' not in self.__file_name:
            pattern = r'(\d{4})_(\w+)_(\w+)_(\w+)\.csv'
        else:
            pattern = r'(\d{4})_(\w+)__(\w+)\.csv'
        self.__filters = re.findall(pattern, self.__file_name)[0]

        self.__correspondents = {
            'EnsinoFundamental': 'EF',
            'AnosIniciais': '1',
            'AnosFinais': '2',
            'TodososValoresdeColunas': 'todos',
            'EnsinoMdio': 'EM',
        }

        self.__year()
        self.__teaching_stage()

    def __year(self):
        self.__year = int(self.__filters[0])

    def __teaching_stage(self):
        self.__teaching_stage = self.__correspondents[self.__filters[1]]
        if self.__teaching_stage == 'EF':
            level = self.__correspondents[self.__filters[2]]
            if level != 'todos':
                self.__teaching_stage += level

    # getters
    def get_year(self):
        return self.__year

    def get_teaching_stage(self):
        return self.__teaching_stage

    def get_file_name(self):
        return self.__file_name

    def get_file_path(self):
        return self.__file_path

    def get_df(self) -> pd.DataFrame:
        return pd.read_csv(self.__file_path, sep=';')

## OFICIAL: Get path from categories and reading and group all file by path

In [4]:
def transform_file_path() -> list:
    source_path_data = os.path.abspath(os.path.join(os.getcwd(), "../oracle_data"))
    category_files = [os.path.join(source_path_data, i) for i in os.listdir(source_path_data)]
    return category_files

all_path_files = transform_file_path()
print(len(all_path_files))


12


## OFICIAL: GEN LIST INSTANCE FILTER BY PATH_FILE

In [5]:
all_category_objs = [InepFilters(i) for i in all_path_files]
print(len(all_category_objs))

12


### TEST: full_table_data (DISPOSABLE)

In [6]:
arquivo1 = all_category_objs[2]

In [7]:
df = arquivo1.get_df()
# df = df.sort_values(by='Categoria 1 - Ordenação')
df.rename(columns={'Categoria 1': 'cor_raca',
                   'Matrículas': 'quantidade',
                   'Categoria 2': 'dependencia_administrativa'}, inplace=True)
df.drop(columns=[
    'Etapa de Ensino - Superior', 'Etapa de Ensino', 'Localidade da Escola', 'Categoria 1 - Ordenação', 'Categoria 2 - Ordenação'], inplace=True)
df['etapa_de_ensino'] = arquivo1.get_teaching_stage()
df['ano'] = arquivo1.get_year()
df['municipio_id'] = df['Município'].map(municipio_map)
display(df)
# verifica se todos os valores da coluna tem tal valor
print((df['UF'] == 'MG').all())

,País,Região,UF,Município,quantidade,cor_raca,dependencia_administrativa,etapa_de_ensino,ano,municipio_id
0,Brasil,Sudeste,MG,Abadia dos Dourados,122,Branca,Estadual,EM,2023,3100104
1,Brasil,Sudeste,MG,Abadia dos Dourados,2,Não declarada,Estadual,EM,2023,3100104
2,Brasil,Sudeste,MG,Abadia dos Dourados,80,Parda,Estadual,EM,2023,3100104
3,Brasil,Sudeste,MG,Abadia dos Dourados,7,Preta,Estadual,EM,2023,3100104
4,Brasil,Sudeste,MG,Abaeté,244,Branca,Estadual,EM,2023,3100203
...,...,...,...,...,...,...,...,...,...,...
4924,Brasil,Sudeste,MG,Volta Grande,82,Parda,Estadual,EM,2023,3172103
4925,Brasil,Sudeste,MG,Volta Grande,29,Preta,Estadual,EM,2023,3172103
4926,Brasil,Sudeste,MG,Wenceslau Braz,70,Branca,Estadual,EM,2023,3172202
4927,Brasil,Sudeste,MG,Wenceslau Braz,36,Parda,Estadual,EM,2023,3172202


True


In [8]:
lines_with_nan = df[df.isna().any(axis=1)]

In [9]:
display(lines_with_nan)

,País,Região,UF,Município,quantidade,cor_raca,dependencia_administrativa,etapa_de_ensino,ano,municipio_id


# OFICIAL CODE:

## Organizing table for insert

In [10]:
def make_full_table(data_filters: InepFilters) -> pd.DataFrame:
    df = data_filters.get_df()
    df.rename(columns={'Categoria 1': 'cor_raca',
                       'Matrículas': 'quantidade',
                       'Categoria 2': 'dependencia_administrativa'}, inplace=True)
    df.drop(columns=[
        'Etapa de Ensino - Superior', 'Etapa de Ensino', 'Localidade da Escola', 'Categoria 1 - Ordenação', 'Categoria 2 - Ordenação'], inplace=True)
    df['etapa_de_ensino'] = data_filters.get_teaching_stage()
    df['ano'] = data_filters.get_year()
    df['municipio_id'] = df['Município'].map(municipio_map)
    return df

## Settings database and create SQLALCHEMY engine

In [11]:
config = dotenv_values("./.env")
username = config.get("DATABASE_USERNAME")
password = config.get("DATABASE_PASSWORD")
dbname = config.get("DATABASE_NAME")
port = config.get("DATABASE_PORT")
host = config.get("DATABASE_HOST")

engine = create_engine(
    f"postgresql+psycopg2://{username}:{password}@{host}:{port}/{dbname}", echo=True)

# inserting in database -> refactore

## necessity apply design patterns: [Design Patterns](https://refactoring.guru/)

## ORM AND CREATE TABLES

In [12]:
class Base(DeclarativeBase):
    ...

In [13]:
class Municipio(Base):
    __tablename__ = 'Municipio'
    id = Column(Integer, primary_key=True, autoincrement=True, default=None)
    nome = Column(String, nullable=False, unique=True)
    UF = Column(String, nullable=False)


class Filtro(Base):
    __tablename__ = 'Filtro'

    id = Column(Integer, primary_key=True, autoincrement=True)
    municipio_id = Column(Integer, ForeignKey('Municipio.id', ondelete='CASCADE', onupdate='CASCADE'), nullable=False)
    etapa_de_ensino = Column(String(5), nullable=False)
    ano = Column(Integer, nullable=False)

    __table_args__ = (UniqueConstraint(
        'municipio_id', 'etapa_de_ensino', 'ano', name='unique_municipio_etapa_ano'),)


class Matricula(Base):
    __tablename__ = 'Matricula'
    id = Column(Integer, primary_key=True, autoincrement=True)
    id_filtro = Column(Integer, ForeignKey('Filtro.id'), nullable=False)
    cor_raca = Column(String(20), nullable=True)
    dependencia_administrativa = Column(String(20), nullable=True)
    quantidade = Column(Integer, nullable=True)

class Indicador(Base):
    __tablename__ = 'Indicador'
    id = Column(Integer, primary_key=True, autoincrement=True)
    id_filtro = Column(Integer, ForeignKey('Filtro.id'), nullable=False)
    dependencia_administrativa = Column(String(20), nullable=True)
    taxa_de_aprovacao = Column(Float, nullable=True)
    taxa_de_reprovacao = Column(Float, nullable=True)
    taxa_de_abandono = Column(Float, nullable=True)


# Configuração do banco de dados
Base.metadata.create_all(engine)

2024-08-28 00:18:17,671 INFO sqlalchemy.engine.Engine select pg_catalog.version()
2024-08-28 00:18:17,679 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-08-28 00:18:18,002 INFO sqlalchemy.engine.Engine select current_schema()
2024-08-28 00:18:18,005 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-08-28 00:18:18,303 INFO sqlalchemy.engine.Engine show standard_conforming_strings
2024-08-28 00:18:18,308 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-08-28 00:18:18,606 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-08-28 00:18:18,616 INFO sqlalchemy.engine.Engine SELECT pg_catalog.pg_class.relname 
FROM pg_catalog.pg_class JOIN pg_catalog.pg_namespace ON pg_catalog.pg_namespace.oid = pg_catalog.pg_class.relnamespace 
WHERE pg_catalog.pg_class.relname = %(table_name)s AND pg_catalog.pg_class.relkind = ANY (ARRAY[%(param_1)s, %(param_2)s, %(param_3)s, %(param_4)s, %(param_5)s]) AND pg_catalog.pg_table_is_visible(pg_catalog.pg_class.oid) AND pg_catalog.pg_namespace.nspname != %(nspname

### Verifica se o filtro existe. Se não existir, cria e retorna o ID

In [14]:
def get_existing_filters():
    # Obtém todos os filtros existentes no banco de dados
    with Session(engine) as session:
        existing_filtros = session.query(Filtro).all()
    # Converte em um dicionário para acesso rápido
    filter_dict = {(f.municipio_id, f.etapa_de_ensino, f.ano): f.id for f in existing_filtros}
    return filter_dict

### Define checkpoints functions

In [15]:
script_path = os.getcwd() # os.path.dirname(os.path.realpath(__file__)) not works in ipynb
CHECKPOINT_FILE = os.path.abspath(os.path.join(script_path, '.data_checkpoint.log'))

In [16]:
def save_checkpoint(file_name):
    with open(CHECKPOINT_FILE, "w") as f:
        f.write(f"{file_name}\n")


def load_checkpoint():
    if os.path.exists(CHECKPOINT_FILE):
        with open(CHECKPOINT_FILE, "r") as f:
            content = f.read().strip()
            if content:
                file_name = content
                return file_name
    return None

In [17]:
def bulk_insert_data(full_table_data: pd.DataFrame, current_file: str=None):
    # Verifica o progresso do checkpoint
    if current_file:
        save_checkpoint(current_file)
    filter_dict = get_existing_filters()
    with Session(engine) as session:
        matricula_objs = []

        # Processar cada linha do DataFrame
        for _, row in full_table_data.iterrows():
            # Obter ou criar o filtro e obter seu ID

            filter_key = (row['municipio_id'],
                          row['etapa_de_ensino'], row['ano'])

            if filter_key in filter_dict:
                # Filtro já existe, usar o ID existente.
                id_filter = filter_dict[filter_key]
            else:
                # Criar novo e adicionar no dicionario
                filtro = Filtro(
                    municipio_id=row['municipio_id'], etapa_de_ensino=row['etapa_de_ensino'], ano=row['ano'])
                session.add(filtro)
                session.flush()
                id_filter = filtro.id
                filter_dict[filter_key] = id_filter

            matricula = Matricula(id_filtro=id_filter,
                                  cor_raca=row['cor_raca'],
                                  dependencia_administrativa=row['dependencia_administrativa'],
                                  quantidade=row['quantidade'])
            matricula_objs.append(matricula)
        # Inserir em lote
        session.bulk_save_objects(matricula_objs)
        session.commit()
    if current_file:
        os.remove(CHECKPOINT_FILE)

In [18]:
def alternate_file_and_insert():
    checkpoint_file = load_checkpoint()
    # garante que a partir de um certo index em all_category_objs, todos serao varridos
    was_not_scanned = False
    if not checkpoint_file:
        was_not_scanned = True
    for matricula_data in all_category_objs:
        if was_not_scanned:
            df_full_table = make_full_table(matricula_data)
            bulk_insert_data(
                df_full_table,
                matricula_data.get_file_path())
        elif checkpoint_file == matricula_data.get_file_path():
            was_not_scanned = True

In [19]:
#alternate_file_and_insert()

## Insert 'Todos' Columns

In [20]:
def check_and_create_all_municipios_id(session : Session):
    # Verificar se o município com ID 1 existe e, se não, inseri-lo
    municipio = session.query(Municipio).filter_by(id=1).first()
    if not municipio:
        municipio = Municipio(id=1, nome='Todos', UF='MG')
        session.add(municipio)
        session.commit()

def municipios_sum_query_df(session: Session):
    query = (
        session.query(
            Filtro.etapa_de_ensino,
            Filtro.ano,
            Matricula.cor_raca,
            Matricula.dependencia_administrativa,
            func.sum(Matricula.quantidade).label('quantidade')
        )
        .join(Matricula, Filtro.id == Matricula.id_filtro)
        .filter(Filtro.municipio_id != 1)
        .filter(Filtro.etapa_de_ensino != 'Todos')
        .group_by(
            Filtro.etapa_de_ensino,
            Filtro.ano,
            Matricula.cor_raca,
            Matricula.dependencia_administrativa
        )
        .order_by(
            Filtro.ano,
            Filtro.etapa_de_ensino,
            Matricula.cor_raca,
            Matricula.dependencia_administrativa
        )
    )

    # Executar a query
    results = query.all()
    # Converter resultados em DataFrame
    df = pd.DataFrame(results, columns=['etapa_de_ensino', 'ano', 'cor_raca', 'dependencia_administrativa', 'quantidade'])
    df['municipio_id'] = 1
    
    display(df)
    return df

def etapa_de_ensino_sum(session: Session):
    query = (
        session.query(
            Filtro.municipio_id,
            Filtro.ano,
            Matricula.cor_raca,
            Matricula.dependencia_administrativa,
            func.sum(Matricula.quantidade).label('quantidade')
        )
        .join(Matricula, Filtro.id == Matricula.id_filtro)
        .filter(Filtro.municipio_id != 1)
        .filter(Filtro.etapa_de_ensino != 'Todos')
        .group_by(
            Filtro.municipio_id,
            Filtro.ano,
            Matricula.cor_raca,
            Matricula.dependencia_administrativa
        )
        .order_by(
            Filtro.ano,
            Filtro.municipio_id,
            Matricula.cor_raca,
            Matricula.dependencia_administrativa
        )
    )
    
    # Executar a query
    results = query.all()
    # Converter resultados em DataFrame
    df = pd.DataFrame(results, columns=['municipio_id', 'ano', 'cor_raca', 'dependencia_administrativa', 'quantidade'])
    df['etapa_de_ensino'] = 'Todos'
    
    display(df)
    return df

def municipios_etapa_de_ensino_sum(session: Session):
    query = (
        session.query(
            Filtro.ano,
            Matricula.cor_raca,
            Matricula.dependencia_administrativa,
            func.sum(Matricula.quantidade).label('quantidade')
        )
        .join(Matricula, Filtro.id == Matricula.id_filtro)
        .filter(Filtro.municipio_id != 1)
        .filter(Filtro.etapa_de_ensino != 'Todos')
        .group_by(
            Filtro.ano,
            Matricula.cor_raca,
            Matricula.dependencia_administrativa
        )
        .order_by(
            Filtro.ano,
            Matricula.cor_raca,
            Matricula.dependencia_administrativa
        )
    )

    # Executar a query
    results = query.all()
    # Converter resultados em DataFrame
    df = pd.DataFrame(results, columns=['ano', 'cor_raca', 'dependencia_administrativa', 'quantidade'])
    df['etapa_de_ensino'] = 'Todos'
    df['municipio_id'] = 1
    
    display(df)
    return df

def send_combination(session: Session):
    check_and_create_all_municipios_id(session=session)
    query_df = municipios_etapa_de_ensino_sum(session=session)
    bulk_insert_data(full_table_data=query_df)

def send_etapa_de_ensino_sum_to_db(session: Session):
    query_df = etapa_de_ensino_sum(session=session)
    bulk_insert_data(full_table_data=query_df)

def send_municipios_sum_to_db(session: Session):
    check_and_create_all_municipios_id(session=session)
    query_df = municipios_sum_query_df(session=session)
    bulk_insert_data(full_table_data=query_df)

def insert_total():
    with Session(engine) as session:
        send_municipios_sum_to_db(session=session)
        send_etapa_de_ensino_sum_to_db(session=session)
        send_combination(session=session)

In [21]:
#insert_total()

# INDICATORS

### SITE FOR SCRAPPER: https://www.gov.br/inep/pt-br/acesso-a-informacao/dados-abertos/indicadores-educacionais

In [22]:
INDICATORS_PATH = os.path.abspath(os.path.join(os.getcwd(), "../indicators_data"))
MUNICIPIOS_FILE_PATH = os.path.join(INDICATORS_PATH, 'tx_rend_municipios_2023.xlsx')
ALL_INDICATORS_FILE_PATH = [os.path.join(INDICATORS_PATH, i) for i in os.listdir(INDICATORS_PATH)]

In [23]:
df_indic = pd.read_excel(ALL_INDICATORS_FILE_PATH[2],
                         skiprows=8,
                         usecols=["NU_ANO_CENSO",
                                  "SG_UF",
                                  "CO_MUNICIPIO",
                                  "NO_MUNICIPIO",
                                  "NO_CATEGORIA",
                                  "NO_DEPENDENCIA",
                                  "1_CAT_FUN_AI",
                                  "1_CAT_FUN_AF",
                                  "1_CAT_MED",
                                  "2_CAT_FUN_AI",
                                  "2_CAT_FUN_AF",
                                  "2_CAT_MED",
                                  "3_CAT_FUN_AI",
                                  "3_CAT_FUN_AF",
                                  "3_CAT_MED"]
                         )

In [24]:
header_map = {'NU_ANO_CENSO':'ano',
             'SG_UF':'UF',
             'CO_MUNICIPIO':'municipio_id',
             'NO_MUNICIPIO':'nome',
             'NO_CATEGORIA':'localização',
             'NO_DEPENDENCIA':'dependencia_administrativa',
             '1_CAT_FUN_AI':'aprovacao_EF1',
             '1_CAT_FUN_AF':'aprovacao_EF2',
             '1_CAT_MED':'aprovacao_EM',
             '2_CAT_FUN_AI':'reprovacao_EF1',
             '2_CAT_FUN_AF':'reprovacao_EF2',
             '2_CAT_MED':'reprovacao_EM',
             '3_CAT_FUN_AI':'abandono_EF1',
             '3_CAT_FUN_AF':'abandono_EF2',
             '3_CAT_MED':'abandono_EM'
            }
df_indic.rename(columns=header_map, inplace=True)

In [25]:
df_indic = df_indic[df_indic['UF'] == 'MG']
df_indic = df_indic[df_indic['localização'] == 'Total']
df_indic = df_indic.drop(columns=['localização'])
df_indic['municipio_id'] = df_indic['municipio_id'].astype(int)

In [26]:
display(df_indic)

,ano,UF,municipio_id,nome,dependencia_administrativa,aprovacao_EF1,aprovacao_EF2,aprovacao_EM,reprovacao_EF1,reprovacao_EF2,reprovacao_EM,abandono_EF1,abandono_EF2,abandono_EM
28427,2023,MG,3100104,Abadia dos Dourados,Total,100,94.1,93.5,0,3.7,1.9,0,2.2,4.6
28430,2023,MG,3100104,Abadia dos Dourados,Estadual,--,94.1,93.5,--,3.7,1.9,--,2.2,4.6
28432,2023,MG,3100104,Abadia dos Dourados,Municipal,100,--,--,0,--,--,0,--,--
28435,2023,MG,3100104,Abadia dos Dourados,Pública,100,94.1,93.5,0,3.7,1.9,0,2.2,4.6
28438,2023,MG,3100203,Abaeté,Total,99.3,84.7,79,0.6,14.7,7.1,0.1,0.6,13.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37884,2023,MG,3172103,Volta Grande,Pública,95.6,88.4,87.7,4.4,11.6,5.4,0,0,6.9
37887,2023,MG,3172202,Wenceslau Braz,Total,96.6,97.7,98.9,3.4,2.3,1.1,0,0,0
37890,2023,MG,3172202,Wenceslau Braz,Estadual,--,97.7,98.9,--,2.3,1.1,--,0,0
37892,2023,MG,3172202,Wenceslau Braz,Municipal,96.6,--,--,3.4,--,--,0,--,--


In [27]:
df_long = pd.melt(
    df_indic,
    id_vars=['ano', 'UF', 'municipio_id', 'nome', 'dependencia_administrativa'],
    value_vars=['aprovacao_EF1', 'aprovacao_EF2', 'aprovacao_EM',
                'reprovacao_EF1', 'reprovacao_EF2', 'reprovacao_EM',
                'abandono_EF1', 'abandono_EF2', 'abandono_EM'],
    var_name='tipo_etapa',
    value_name='valor'
)

In [28]:
df_long[['tipo', 'etapa_de_ensino']] = df_long['tipo_etapa'].str.split('_', expand=True)
df_long.drop('tipo_etapa', axis=1, inplace=True)

In [29]:
df_pivot = df_long.pivot_table(
    index=['ano', 'UF', 'municipio_id', 'nome', 'dependencia_administrativa', 'etapa_de_ensino'],
    columns='tipo',
    values='valor',
    aggfunc='first'
).reset_index()

In [30]:
import numpy as np

df_pivot.columns.name = None
df_pivot.rename(columns={'aprovacao': 'taxa_de_aprovacao', 'reprovacao': 'taxa_de_reprovacao', 'abandono': 'taxa_de_abandono', 'nome':'Município'}, inplace=True)
with pd.option_context('future.no_silent_downcasting', True):
    df_pivot['taxa_de_aprovacao'] = df_pivot['taxa_de_aprovacao'].replace({'--': np.nan})
    df_pivot['taxa_de_reprovacao'] = df_pivot['taxa_de_reprovacao'].replace({'--': np.nan})
    df_pivot['taxa_de_abandono'] = df_pivot['taxa_de_abandono'].replace({'--': np.nan})

df_pivot = df_pivot.where(pd.notnull(df_pivot), None) # substitui NaN por None
display(df_pivot)
print(df_pivot['taxa_de_aprovacao'].dtype)

,ano,UF,municipio_id,Município,dependencia_administrativa,etapa_de_ensino,taxa_de_abandono,taxa_de_aprovacao,taxa_de_reprovacao
0,2023,MG,3100104,Abadia dos Dourados,Estadual,EF1,None,None,None
1,2023,MG,3100104,Abadia dos Dourados,Estadual,EF2,2.2,94.1,3.7
2,2023,MG,3100104,Abadia dos Dourados,Estadual,EM,4.6,93.5,1.9
3,2023,MG,3100104,Abadia dos Dourados,Municipal,EF1,0,100,0
4,2023,MG,3100104,Abadia dos Dourados,Municipal,EF2,None,None,None
...,...,...,...,...,...,...,...,...,...
11305,2023,MG,3172202,Wenceslau Braz,Pública,EF2,0,97.7,2.3
11306,2023,MG,3172202,Wenceslau Braz,Pública,EM,0,98.9,1.1
11307,2023,MG,3172202,Wenceslau Braz,Total,EF1,0,96.6,3.4
11308,2023,MG,3172202,Wenceslau Braz,Total,EF2,0,97.7,2.3


object


In [41]:
def make_full_table2(indicator_file_path) -> pd.DataFrame:
    df_indic = pd.read_excel(indicator_file_path,
                         skiprows=8,
                         usecols=["NU_ANO_CENSO",
                                  "SG_UF",
                                  "CO_MUNICIPIO",
                                  "NO_MUNICIPIO",
                                  "NO_CATEGORIA",
                                  "NO_DEPENDENCIA",
                                  "1_CAT_FUN_AI",
                                  "1_CAT_FUN_AF",
                                  "1_CAT_MED",
                                  "2_CAT_FUN_AI",
                                  "2_CAT_FUN_AF",
                                  "2_CAT_MED",
                                  "3_CAT_FUN_AI",
                                  "3_CAT_FUN_AF",
                                  "3_CAT_MED"]
                         )
    header_map = {'NU_ANO_CENSO':'ano',
                'SG_UF':'UF',
                'CO_MUNICIPIO':'municipio_id',
                'NO_MUNICIPIO':'nome',
                'NO_CATEGORIA':'localização',
                'NO_DEPENDENCIA':'dependencia_administrativa',
                '1_CAT_FUN_AI':'aprovacao_EF1',
                '1_CAT_FUN_AF':'aprovacao_EF2',
                '1_CAT_MED':'aprovacao_EM',
                '2_CAT_FUN_AI':'reprovacao_EF1',
                '2_CAT_FUN_AF':'reprovacao_EF2',
                '2_CAT_MED':'reprovacao_EM',
                '3_CAT_FUN_AI':'abandono_EF1',
                '3_CAT_FUN_AF':'abandono_EF2',
                '3_CAT_MED':'abandono_EM'
                }
    df_indic.rename(columns=header_map, inplace=True)
    df_indic = df_indic[df_indic['UF'] == 'MG']
    df_indic = df_indic[df_indic['localização'] == 'Total']
    df_indic = df_indic.drop(columns=['localização'])
    df_indic['municipio_id'] = df_indic['municipio_id'].astype(int)
    df_long = pd.melt(
        df_indic,
        id_vars=['ano', 'UF', 'municipio_id', 'nome', 'dependencia_administrativa'],
        value_vars=['aprovacao_EF1', 'aprovacao_EF2', 'aprovacao_EM',
                    'reprovacao_EF1', 'reprovacao_EF2', 'reprovacao_EM',
                    'abandono_EF1', 'abandono_EF2', 'abandono_EM'],
        var_name='tipo_etapa',
        value_name='valor'
    )
    df_long[['tipo', 'etapa_de_ensino']] = df_long['tipo_etapa'].str.split('_', expand=True)
    df_long.drop('tipo_etapa', axis=1, inplace=True)
    df_pivot = df_long.pivot_table(
        index=['ano', 'UF', 'municipio_id', 'nome', 'dependencia_administrativa', 'etapa_de_ensino'],
        columns='tipo',
        values='valor',
        aggfunc='first'
    ).reset_index()
    df_pivot.columns.name = None
    df_pivot.rename(columns={'aprovacao': 'taxa_de_aprovacao', 'reprovacao': 'taxa_de_reprovacao', 'abandono': 'taxa_de_abandono', 'nome':'Município'}, inplace=True)
    with pd.option_context('future.no_silent_downcasting', True):
        df_pivot['taxa_de_aprovacao'] = df_pivot['taxa_de_aprovacao'].replace({'--': np.nan})
        df_pivot['taxa_de_reprovacao'] = df_pivot['taxa_de_reprovacao'].replace({'--': np.nan})
        df_pivot['taxa_de_abandono'] = df_pivot['taxa_de_abandono'].replace({'--': np.nan})

    df_pivot = df_pivot.where(pd.notnull(df_pivot), None) # substitui NaN por None
    return df_pivot

In [42]:
def bulk_insert_data2(full_table_data: pd.DataFrame, current_file: str=None):
    # Verifica o progresso do checkpoint
    if current_file:
        save_checkpoint(current_file)
    filter_dict = get_existing_filters()
    with Session(engine) as session:
        indicador_objs = []

        # Processar cada linha do DataFrame
        for _, row in full_table_data.iterrows():
            # Obter ou criar o filtro e obter seu ID

            filter_key = (row['municipio_id'],
                          row['etapa_de_ensino'], row['ano'])

            if filter_key in filter_dict:
                # Filtro já existe, usar o ID existente.
                id_filter = filter_dict[filter_key]
            else:
                # Criar novo e adicionar no dicionario
                filtro = Filtro(
                    municipio_id=row['municipio_id'], etapa_de_ensino=row['etapa_de_ensino'], ano=row['ano'])
                session.add(filtro)
                session.flush()
                id_filter = filtro.id
                filter_dict[filter_key] = id_filter

            indicador = Indicador(id_filtro=id_filter,
                                  dependencia_administrativa=row['dependencia_administrativa'],
                                  taxa_de_aprovacao=row['taxa_de_aprovacao'],
                                  taxa_de_reprovacao=row['taxa_de_reprovacao'],
                                  taxa_de_abandono=row['taxa_de_abandono'])
            indicador_objs.append(indicador)
        # Inserir em lote
        session.bulk_save_objects(indicador_objs)
        session.commit()
    if current_file:
        os.remove(CHECKPOINT_FILE)

In [33]:
#bulk_insert_data2(df_pivot)

In [ ]:
def alternate_file_and_insert2(list_file_path: list):
    checkpoint_file = load_checkpoint()
    # garante que a partir de um certo index em all_category_objs, todos serao varridos
    was_not_scanned = False
    if not checkpoint_file:
        was_not_scanned = True
    for file_path in list_file_path:
        if was_not_scanned:
            df_full_table = make_full_table2(file_path)
            display(df_full_table)
            bulk_insert_data2(
                df_full_table,
                file_path)
        elif checkpoint_file == file_path:
            was_not_scanned = True

In [46]:
alternate_file_and_insert2(ALL_INDICATORS_FILE_PATH)

Exception ignored in: <function ZipFile.__del__ at 0x760003f2b2e0>
Traceback (most recent call last):
  File "/home/jorranico/miniconda3/envs/WebScrapper/lib/python3.12/zipfile/__init__.py", line 1938, in __del__
    def __del__(self):

KeyboardInterrupt: 


,ano,UF,municipio_id,Município,dependencia_administrativa,etapa_de_ensino,taxa_de_abandono,taxa_de_aprovacao,taxa_de_reprovacao
0,2022,MG,3100104,Abadia dos Dourados,Estadual,EF1,None,None,None
1,2022,MG,3100104,Abadia dos Dourados,Estadual,EF2,0.7,93.2,6.1
2,2022,MG,3100104,Abadia dos Dourados,Estadual,EM,2,91.6,6.4
3,2022,MG,3100104,Abadia dos Dourados,Municipal,EF1,0,100,0
4,2022,MG,3100104,Abadia dos Dourados,Municipal,EF2,None,None,None
...,...,...,...,...,...,...,...,...,...
11308,2022,MG,3172202,Wenceslau Braz,Pública,EF2,0,96.8,3.2
11309,2022,MG,3172202,Wenceslau Braz,Pública,EM,0,97.8,2.2
11310,2022,MG,3172202,Wenceslau Braz,Total,EF1,0,100,0
11311,2022,MG,3172202,Wenceslau Braz,Total,EF2,0,96.8,3.2


2024-08-28 00:48:05,642 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-08-28 00:48:05,643 INFO sqlalchemy.engine.Engine SELECT "Filtro".id AS "Filtro_id", "Filtro".municipio_id AS "Filtro_municipio_id", "Filtro".etapa_de_ensino AS "Filtro_etapa_de_ensino", "Filtro".ano AS "Filtro_ano" 
FROM "Filtro"
2024-08-28 00:48:05,644 INFO sqlalchemy.engine.Engine [cached since 1710s ago] {}
2024-08-28 00:48:05,944 INFO sqlalchemy.engine.Engine ROLLBACK
2024-08-28 00:48:06,096 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-08-28 00:48:06,099 INFO sqlalchemy.engine.Engine INSERT INTO "Filtro" (municipio_id, etapa_de_ensino, ano) VALUES (%(municipio_id)s, %(etapa_de_ensino)s, %(ano)s) RETURNING "Filtro".id
2024-08-28 00:48:06,104 INFO sqlalchemy.engine.Engine [cached since 1562s ago] {'municipio_id': 3100104, 'etapa_de_ensino': 'EF1', 'ano': 2022}
2024-08-28 00:48:06,406 INFO sqlalchemy.engine.Engine INSERT INTO "Filtro" (municipio_id, etapa_de_ensino, ano) VALUES (%(municipio_id)s, %(etap


KeyboardInterrupt



## Insert Todos


In [36]:
def municipios2_sum_query_df(session: Session):
    query = (
        session.query(
            Filtro.etapa_de_ensino,
            Filtro.ano,
            Indicador.dependencia_administrativa,
            func.avg(Indicador.taxa_de_aprovacao).label('taxa_de_aprovacao'),
            func.avg(Indicador.taxa_de_reprovacao).label('taxa_de_reprovacao'),
            func.avg(Indicador.taxa_de_abandono).label('taxa_de_abandono')
        )
        .join(Indicador, Filtro.id == Indicador.id_filtro)
        .filter(Filtro.municipio_id != 1)
        .filter(Filtro.etapa_de_ensino != 'Todos')
        .group_by(
            Filtro.etapa_de_ensino,
            Filtro.ano,
            Indicador.dependencia_administrativa
        )
        .order_by(
            Filtro.ano,
            Filtro.etapa_de_ensino,
            Indicador.dependencia_administrativa
        )
    )

    # Executar a query
    results = query.all()
    # Converter resultados em DataFrame
    df = pd.DataFrame(results,
                      columns=[
                          'etapa_de_ensino',
                          'ano',
                          'dependencia_administrativa',
                          'taxa_de_aprovacao',
                          'taxa_de_reprovacao',
                          'taxa_de_abandono'
                          ])
    df['municipio_id'] = 1
    
    display(df)
    return df

def etapa_de_ensino2_sum(session: Session):
    query = (
        session.query(
            Filtro.municipio_id,
            Filtro.ano,
            Indicador.dependencia_administrativa,
            func.avg(Indicador.taxa_de_aprovacao).label('taxa_de_aprovacao'),
            func.avg(Indicador.taxa_de_reprovacao).label('taxa_de_reprovacao'),
            func.avg(Indicador.taxa_de_abandono).label('taxa_de_abandono')
        )
        .join(Indicador, Filtro.id == Indicador.id_filtro)
        .filter(Filtro.municipio_id != 1)
        .filter(Filtro.etapa_de_ensino != 'Todos')
        .group_by(
            Filtro.municipio_id,
            Filtro.ano,
            Indicador.dependencia_administrativa
        )
        .order_by(
            Filtro.ano,
            Filtro.municipio_id,
            Indicador.dependencia_administrativa
        )
    )
    
    # Executar a query
    results = query.all()
    # Converter resultados em DataFrame
    df = pd.DataFrame(results,
                      columns=[
                          'municipio_id',
                          'ano',
                          'dependencia_administrativa',
                          'taxa_de_aprovacao',
                          'taxa_de_reprovacao',
                          'taxa_de_abandono'
                          ])
    df['etapa_de_ensino'] = 'Todos'
    
    display(df)
    return df

def municipios_etapa_de_ensino2_sum(session: Session):
    query = (
        session.query(
            Filtro.ano,
            Indicador.dependencia_administrativa,
            func.avg(Indicador.taxa_de_aprovacao).label('taxa_de_aprovacao'),
            func.avg(Indicador.taxa_de_reprovacao).label('taxa_de_reprovacao'),
            func.avg(Indicador.taxa_de_abandono).label('taxa_de_abandono')
        )
        .join(Indicador, Filtro.id == Indicador.id_filtro)
        .filter(Filtro.municipio_id != 1)
        .filter(Filtro.etapa_de_ensino != 'Todos')
        .group_by(
            Filtro.ano,
            Indicador.dependencia_administrativa
        )
        .order_by(
            Filtro.ano,
            Indicador.dependencia_administrativa
        )
    )

    # Executar a query
    results = query.all()
    # Converter resultados em DataFrame
    df = pd.DataFrame(results,
                      columns=[
                          'ano',
                          'dependencia_administrativa',
                          'taxa_de_aprovacao',
                          'taxa_de_reprovacao',
                          'taxa_de_abandono'
                          ])
    df['etapa_de_ensino'] = 'Todos'
    df['municipio_id'] = 1
    
    display(df)
    return df

def send_combination2(session: Session):
    check_and_create_all_municipios_id(session=session)
    query_df = municipios_etapa_de_ensino2_sum(session=session)
    bulk_insert_data2(full_table_data=query_df)

def send_etapa_de_ensino_sum_to_db2(session: Session):
    query_df = etapa_de_ensino2_sum(session=session)
    bulk_insert_data2(full_table_data=query_df)

def send_municipios_sum_to_db2(session: Session):
    check_and_create_all_municipios_id(session=session)
    query_df = municipios2_sum_query_df(session=session)
    bulk_insert_data2(full_table_data=query_df)

def insert_total2():
    with Session(engine) as session:
        send_municipios_sum_to_db2(session=session)
        send_etapa_de_ensino_sum_to_db2(session=session)
        send_combination2(session=session)

In [37]:
insert_total2()

2024-08-28 00:19:33,189 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-08-28 00:19:33,196 INFO sqlalchemy.engine.Engine SELECT "Municipio".id AS "Municipio_id", "Municipio".nome AS "Municipio_nome", "Municipio"."UF" AS "Municipio_UF" 
FROM "Municipio" 
WHERE "Municipio".id = %(id_1)s 
 LIMIT %(param_1)s
2024-08-28 00:19:33,197 INFO sqlalchemy.engine.Engine [generated in 0.00116s] {'id_1': 1, 'param_1': 1}


2024-08-28 00:19:33,507 INFO sqlalchemy.engine.Engine INSERT INTO "Municipio" (id, nome, "UF") VALUES (%(id)s, %(nome)s, %(UF)s)
2024-08-28 00:19:33,509 INFO sqlalchemy.engine.Engine [generated in 0.00178s] {'id': 1, 'nome': 'Todos', 'UF': 'MG'}
2024-08-28 00:19:33,665 INFO sqlalchemy.engine.Engine COMMIT
2024-08-28 00:19:33,820 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-08-28 00:19:33,826 INFO sqlalchemy.engine.Engine SELECT "Filtro".etapa_de_ensino AS "Filtro_etapa_de_ensino", "Filtro".ano AS "Filtro_ano", "Indicador".dependencia_administrativa AS "Indicador_dependencia_administrativa", avg("Indicador".taxa_de_aprovacao) AS taxa_de_aprovacao, avg("Indicador".taxa_de_reprovacao) AS taxa_de_reprovacao, avg("Indicador".taxa_de_abandono) AS taxa_de_abandono 
FROM "Filtro" JOIN "Indicador" ON "Filtro".id = "Indicador".id_filtro 
WHERE "Filtro".municipio_id != %(municipio_id_1)s AND "Filtro".etapa_de_ensino != %(etapa_de_ensino_1)s GROUP BY "Filtro".etapa_de_ensino, "Filtro".ano, 

,etapa_de_ensino,ano,dependencia_administrativa,taxa_de_aprovacao,taxa_de_reprovacao,taxa_de_abandono,municipio_id


2024-08-28 00:19:35,193 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-08-28 00:19:35,196 INFO sqlalchemy.engine.Engine SELECT "Filtro".id AS "Filtro_id", "Filtro".municipio_id AS "Filtro_municipio_id", "Filtro".etapa_de_ensino AS "Filtro_etapa_de_ensino", "Filtro".ano AS "Filtro_ano" 
FROM "Filtro"
2024-08-28 00:19:35,197 INFO sqlalchemy.engine.Engine [generated in 0.00112s] {}
2024-08-28 00:19:35,501 INFO sqlalchemy.engine.Engine ROLLBACK
2024-08-28 00:19:35,656 INFO sqlalchemy.engine.Engine SELECT "Filtro".municipio_id AS "Filtro_municipio_id", "Filtro".ano AS "Filtro_ano", "Indicador".dependencia_administrativa AS "Indicador_dependencia_administrativa", avg("Indicador".taxa_de_aprovacao) AS taxa_de_aprovacao, avg("Indicador".taxa_de_reprovacao) AS taxa_de_reprovacao, avg("Indicador".taxa_de_abandono) AS taxa_de_abandono 
FROM "Filtro" JOIN "Indicador" ON "Filtro".id = "Indicador".id_filtro 
WHERE "Filtro".municipio_id != %(municipio_id_1)s AND "Filtro".etapa_de_ensino != %(eta

,municipio_id,ano,dependencia_administrativa,taxa_de_aprovacao,taxa_de_reprovacao,taxa_de_abandono,etapa_de_ensino


2024-08-28 00:19:35,812 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-08-28 00:19:35,813 INFO sqlalchemy.engine.Engine SELECT "Filtro".id AS "Filtro_id", "Filtro".municipio_id AS "Filtro_municipio_id", "Filtro".etapa_de_ensino AS "Filtro_etapa_de_ensino", "Filtro".ano AS "Filtro_ano" 
FROM "Filtro"
2024-08-28 00:19:35,814 INFO sqlalchemy.engine.Engine [cached since 0.6183s ago] {}
2024-08-28 00:19:36,115 INFO sqlalchemy.engine.Engine ROLLBACK
2024-08-28 00:19:36,266 INFO sqlalchemy.engine.Engine SELECT "Municipio".id AS "Municipio_id", "Municipio".nome AS "Municipio_nome", "Municipio"."UF" AS "Municipio_UF" 
FROM "Municipio" 
WHERE "Municipio".id = %(id_1)s 
 LIMIT %(param_1)s
2024-08-28 00:19:36,267 INFO sqlalchemy.engine.Engine [cached since 3.072s ago] {'id_1': 1, 'param_1': 1}
2024-08-28 00:19:36,426 INFO sqlalchemy.engine.Engine SELECT "Filtro".ano AS "Filtro_ano", "Indicador".dependencia_administrativa AS "Indicador_dependencia_administrativa", avg("Indicador".taxa_de_aprov

,ano,dependencia_administrativa,taxa_de_aprovacao,taxa_de_reprovacao,taxa_de_abandono,etapa_de_ensino,municipio_id


2024-08-28 00:19:36,592 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-08-28 00:19:36,593 INFO sqlalchemy.engine.Engine SELECT "Filtro".id AS "Filtro_id", "Filtro".municipio_id AS "Filtro_municipio_id", "Filtro".etapa_de_ensino AS "Filtro_etapa_de_ensino", "Filtro".ano AS "Filtro_ano" 
FROM "Filtro"
2024-08-28 00:19:36,594 INFO sqlalchemy.engine.Engine [cached since 1.398s ago] {}
2024-08-28 00:19:36,921 INFO sqlalchemy.engine.Engine ROLLBACK
2024-08-28 00:19:37,076 INFO sqlalchemy.engine.Engine ROLLBACK
